In [1]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline0724

/content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline0724


In [4]:
!pip install fmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 9.1 MB/s eta 0:00:00


In [5]:
from fmpy import read_model_description, extract
from fmpy.fmi2 import FMU2Slave
from fmpy.util import plot_result, download_test_file
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt

import random
import numpy as np
import torch
import argparse
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras

from collections import deque
import numpy as np
from tqdm import tqdm

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import scipy.signal
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch import FloatTensor as FT

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
from torch.utils.tensorboard import SummaryWriter

In [6]:
fmu_filename = 'HEV_TMED_Simulator_Rearrange_230711_linux.fmu'
start_time = 0.0
stop_time = 1800.0
step_size = 1
soc_init = 67

# WLTP profile에 대하여 시험
with open("wltp_vehicle_speed_profile_real.csv") as file_name:
    vehicle_speed_profile = np.loadtxt(file_name, delimiter=",")

EngLine = pd.read_csv('wltp_EngLine_rule_based_10Hz.csv')
EngLine = np.array(EngLine['EngLine'])

In [7]:
class HEV:
    def __init__(self, fmu_filename, start_time, stop_time, step_size, soc_init, vehicle_speed_profile):
        self.fmu_filename = fmu_filename
        self.vrs = {}
        self.start_time = start_time
        self.stop_time = stop_time
        self.step_size = step_size
        self.time = self.start_time
        self.vehicle_speed_profile = vehicle_speed_profile
        self.soc_init = soc_init/5000
        self.soc_base = soc_init/5000
        self.state_init = np.array([self.soc_init, 0, 0]).reshape(1,-1)
        self.state = np.array([self.soc_init, 0, 0]).reshape(1,-1)
        self.action_upper_bound = 15000
        self.action_lower_bound = -15000
        #self.action_space = [[13500, 2000], [500, 2000], [13500, -15000], [500, -15000]]
        self.obssize = len(self.state[0])
        self.actsize = 2

        model_description = read_model_description(self.fmu_filename)
        for variable in model_description.modelVariables:
            self.vrs[variable.name] = variable.valueReference
        unzipdir = extract(fmu_filename)
        self.fmu = FMU2Slave(guid=model_description.guid,
                       unzipDirectory=unzipdir,
                       modelIdentifier=model_description.coSimulation.modelIdentifier,
                       instanceName='instance1')

    def step(self, action):
        #a = self.action_space[action]
#         action = action*self.action_upper_bound
        a1 = action[0]*self.action_upper_bound
        a2 = action[0]*self.action_upper_bound
        a3 = self.soc_init*5000
        a4 = action[1]/2 + 1
        instant_veh_speed = np.interp(self.time, self.vehicle_speed_profile[:,0], self.vehicle_speed_profile[:,1])
        self.fmu.setReal([self.vr_input1, self.vr_input2, self.vr_input3, self.vr_input4, self.vr_input5], [instant_veh_speed, a1, a2, a3, a4]) #input variable, input key(13500 2000)
        self.fmu.doStep(currentCommunicationPoint=self.time, communicationStepSize=self.step_size)
#         [output1, output2, output3, output4, output5, output6, output7] = self.fmu.getReal([self.vr_output1,self.vr_output2, self.vr_output3, self.vr_output4, self.vr_output5, self.vr_output6, self.vr_output7])
#         self.state = [output1, output2, output3, output4, output5, output6, output7]
        state = np.array(self.fmu.getReal(np.arange(35)))/5000
        state_column = np.array([self.vrs['Bat_SOC'], self.vrs['BSFC_g_kWh[1]'], self.vrs['Driver_sVeh_kph']])
        self.state = state[state_column]
        soc = state[self.vrs['Bat_SOC']]
        BSFC = state[self.vrs['BSFC_g_kWh[1]']]
        reward = 1 - 10000*(self.soc_init - soc) ** 2 - BSFC
        is_done = lambda time: time >= self.stop_time
        info = state[np.array([self.vrs['Bat_SOC'], self.vrs['nEngOn'], self.vrs['PT_tqTmInDmd_Nm_P2'], self.vrs['ObEng_nEng_Rpm'], self.vrs['TrEtp_tqEngMAF_Nm'], self.vrs['TrP0_tqP0_Nm'], self.vrs['TrP2_tqP2_Nm'], self.vrs['Driver_sVeh_kph'], self.vrs['BSFC_g_kWh[1]']])]
        self.time += self.step_size
        return self.state.reshape(1,-1), reward, is_done(self.time), info

    def reset(self):

        self.fmu.instantiate()
        self.fmu.setupExperiment(startTime=self.start_time)
        self.fmu.enterInitializationMode()
        self.fmu.exitInitializationMode()
        self.state = self.state_init
        self.time = self.start_time

        self.vr_input1 = self.vrs['Driver_sVeh_Target_kph']
        self.vr_input2 = self.vrs['Engine_on_line']
        self.vr_input3 = self.vrs['Engine_off_line']
        self.vr_input4 = self.vrs['soc_init']
        self.vr_input5 = self.vrs['Engine_OOL']
        self.vr_output1 = self.vrs['TgMod_fPt']
        self.vr_output2 = self.vrs['P2_wElec_W']
        self.vr_output3 = self.vrs['P4_wElec_W']
        self.vr_output4 = self.vrs['ObP2_wElecBIntv_W']
        self.vr_output5 = self.vrs['EV_on_line']
        self.vr_output6 = self.vrs['EV_off_line']
        self.vr_output7 = self.vrs['Bat_SOC']
        self.vr_output8 = self.vrs['PT_tqTmInDmd_Nm_P2']
        self.vr_output9 = self.vrs['P0_wElec_W']
        self.vr_output10 = self.vrs['Pwr_Aux_W']
        self.vr_output11 = self.vrs['ObEng_nEng_Rpm']
        self.vr_output12 = self.vrs['TrEtp_tqEngMAF_Nm']
        self.vr_output13 = self.vrs['rpm_P0']
        self.vr_output14 = self.vrs['TM_F_nTmIn_rpm']
        self.vr_output15 = self.vrs['TM_R_nTmIn_rpm']
        self.vr_output16 = self.vrs['eBat_kWh']
        self.vr_output17 = self.vrs['nEngOn']
        self.vr_output18 = self.vrs['TrP0_tqP0_Nm']
        self.vr_output19 = self.vrs['TrP2_tqP2_Nm']
        self.vr_output20 = self.vrs['TrP4_tqP4_Nm']
        self.vr_output21 = self.vrs['TCU_F_fCurGe']
        self.vr_output22 = self.vrs['TCU_F_fTarGe']
        self.vr_output23 = self.vrs['Driver_sVeh_kph']
        self.vr_output24 = self.vrs['Eng_eff_avg']
        self.vr_output25 = self.vrs['TM_F_P0P2_eff_avg']
        self.vr_output26 = self.vrs['TM_F_P0P4_eff_avg']
        self.vr_output27 = self.vrs['BSFC_g_kWh[1]']
        self.vr_output28 = self.vrs['BSFC_g_kWh[2]']
        self.vr_output29 = self.vrs['BSFC_g_kWh[3]']

        return self.state.reshape(1,-1)

In [8]:
env = HEV(fmu_filename, start_time, stop_time, step_size, soc_init, vehicle_speed_profile)
s = env.reset()
num_states, num_actions = 3, 2

upper_bound = env.action_upper_bound/env.action_upper_bound
lower_bound = env.action_lower_bound/env.action_upper_bound

print("Max Value of Action ->  {}".format(upper_bound))
print("Min Value of Action ->  {}".format(lower_bound))

Max Value of Action ->  1.0
Min Value of Action ->  -1.0


In [9]:
class Buffer:
    def __init__(self, buffer_capacity, batch_size, tau, policy_freq, policy_noise, noise_clip):
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.iteration = 0
        self.policy_freq = policy_freq
        self.buffer_counter = 0
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip
        self.tau = tau

        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))

    def record(self, obs_tuple):
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        self.buffer_counter += 1

    def learn(self):
        # Get sampling range
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        # self.update(state_batch, action_batch, reward_batch, next_state_batch)

        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            noise = tf.clip_by_value(tf.random.normal(action.shape) * self.policy_noise, -self.noise_clip, self.noise_clip)
            next_state_action = tf.clip_by_value(target_actor(next_state_batch, training = True)+noise, lower_bound, upper_bound)
            next_state_batch = tf.cast(next_state_batch, dtype=tf.float32)
            a=tf.concat([next_state_batch, next_state_action], axis = 1)
            target_q1 = target_critic1(a, training = True)
            target_q2 = target_critic2(a, training = True)
            target_q = tf.reduce_min([target_q1, target_q2], axis = 0)
            target_q = reward_batch + gamma * target_q

            q1 = critic1(tf.concat([state_batch,action_batch],axis=1), training = True)
            q2 = critic2(tf.concat([state_batch,action_batch],axis=1), training = True)
            target_q = tf.cast(target_q, dtype = tf.float32)
            qf1_loss = tf.reduce_mean(tf.math.square(q1-target_q))
            qf2_loss = tf.reduce_mean(tf.math.square(q2-target_q))
            qf_loss = qf1_loss + qf2_loss
        critic1_gradients = tape1.gradient(qf_loss, critic1.trainable_variables)
        critic1_optimizer.apply_gradients(zip(critic1_gradients, critic1.trainable_variables))
        critic2_gradients = tape2.gradient(qf_loss, critic2.trainable_variables)
        critic2_optimizer.apply_gradients(zip(critic2_gradients, critic2.trainable_variables))


        if self.iteration % self.policy_freq == 0:
          with tf.GradientTape() as tape3:
              # print('-----------------------------------------------------------')
              # print('actor update')
              actions = actor(state_batch, training=True)
              state_batch = tf.cast(state_batch, dtype=tf.float32)
              critic_value = critic1(tf.concat([state_batch, actions],axis=1), training=True)
              actor_loss = -tf.math.reduce_mean(critic_value)

          actor_gradients = tape3.gradient(actor_loss, actor.trainable_variables)
          actor_optimizer.apply_gradients(zip(actor_gradients, actor.trainable_variables))
          # print(actor_gradients)

          update_target(target_critic1.variables, critic1.variables, self.tau)
          update_target(target_critic2.variables, critic2.variables, self.tau)
          update_target(target_actor.variables, actor.variables, self.tau)

        self.iteration += 1

# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
@tf.function
def update_target(target_weights, weights, tau):
    for (a, b) in zip(target_weights, weights):
        a.assign(b * tau + a * (1 - tau))

In [10]:
last_init = tf.random_uniform_initializer(minval=-0.1, maxval=0.1)
def get_actor():
    inputs = layers.Input(shape=(num_states,))
    out = layers.Dense(256, activation="relu")(inputs)
    out = layers.Dense(256, activation="relu")(out)
    outputs = layers.Dense(2, activation="tanh", kernel_initializer=last_init)(out)

    model = tf.keras.Model(inputs, outputs)
    return model


def get_critic():
    # # State as input
    # state_input = layers.Input(shape=(num_states))
    # state_out = layers.Dense(16, activation="relu")(state_input)
    # state_out = layers.Dense(32, activation="relu")(state_out)

    # # Action as input
    # action_input = layers.Input(shape=(num_actions))
    # action_out = layers.Dense(32, activation="relu")(action_input)

    # # Both are passed through seperate layer before concatenating
    # concat = layers.Concatenate()([state_out, action_out])
    Input = layers.Input(shape = (num_states+num_actions))
    out1 = layers.Dense(256, activation="relu")(Input)
    out2 = layers.Dense(256, activation="relu")(out1)
    outputs = layers.Dense(1,kernel_initializer=last_init)(out2)
    # Outputs single value for give state-action
    model = tf.keras.Model(Input, outputs)
    return model

In [11]:
actor = get_actor()
target_actor = get_actor()

critic1 = get_critic()
critic2 = get_critic()
target_critic1 = get_critic()
target_critic2 = get_critic()

# Making the weights equal initially
target_actor.set_weights(actor.get_weights())
target_critic1.set_weights(critic1.get_weights())
target_critic2.set_weights(critic2.get_weights())

# Learning rate for actor-critic models
critic_lr = 0.002
actor_lr = 0.001

critic1_optimizer = tf.keras.optimizers.Adam(critic_lr)
critic2_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

# Discount factor for future rewards
gamma = 0.99

buffer = Buffer(50000, 64, 0.01, 3, 0.2, 0.5) # buffer size, batch size, tau, policy_update_frequency, policy_noise, noise_clip

In [12]:
ep_reward_list = []

env = HEV(fmu_filename, start_time, stop_time, step_size, soc_init, vehicle_speed_profile)
start = time.time()

for ep in range(500):

    state = env.reset()
    episodic_reward = 0
    i = 0
    while True:
        action = actor(state)
        # print('----------------------------------------------------------------')
        # print(action)
        next_state, reward, done, info = env.step(action[0].numpy())

        buffer.record((state, action, reward, next_state))
        episodic_reward += reward

        if i%20==ep%20:
          buffer.learn()

        if done:
            break

        state = next_state
        i += 1
    print('---------------------------------------------------------------')
    print('ep     : {}'.format(ep))
    print('reward : {}'.format(episodic_reward))
    print('time   : {}'.format(time.time()-start))
    ep_reward_list.append(episodic_reward)
    if (ep+1) % 10 == 0:
      print('model and reward saved!!!!!!!!!!!!!!!!!')
      actor.save_weights("/content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline0724/weight/actor_1.h5")
      np.save('/content/drive/MyDrive/Colab Notebooks/기계시스템설계/23년/baseline0724/ddpg_reward/2_r.npy', np.array(ep_reward_list))

---------------------------------------------------------------
ep     : 0
reward : 1502.599817268881
time   : 17.113231897354126
---------------------------------------------------------------
ep     : 1
reward : 726.928751754931
time   : 33.22098231315613
---------------------------------------------------------------
ep     : 2
reward : 1576.3287077171296
time   : 49.628007888793945
---------------------------------------------------------------
ep     : 3
reward : 1405.5797357635445
time   : 68.01826000213623
---------------------------------------------------------------
ep     : 4
reward : 520.0342052579124
time   : 83.86496162414551
---------------------------------------------------------------
ep     : 5
reward : 443.9889513171498
time   : 99.56458783149719
---------------------------------------------------------------
ep     : 6
reward : 444.09961114088117
time   : 115.53866267204285
---------------------------------------------------------------
ep     : 7
reward : 441.2997